In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
from random import randint
import random
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [3]:
def parse_model_data(DataFrame,feature):
    train_xy = pd.read_csv(DataFrame)
    train_xy.drop("Unnamed: 0",axis=1,inplace=True)
    train, val = train_test_split(train_xy, test_size=0.2, random_state=20)
    train_y = train[feature]
    train_x = train.drop(feature, axis=1)
    val_y = val[feature]
    val_x = val.drop(feature, axis=1)
    return train_x, train_y, val_x, val_y
train_x, train_y, val_x, val_y = parse_model_data("20_fea_model_data.csv","pIC50")

<IPython.core.display.Javascript object>

In [4]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / (y_true +1e-4))) * 100

# XG 预测

In [17]:
def xgb_reg():
    model_reg = xgb.XGBRegressor()
    model_reg.fit(train_x, train_y)
    predict_y = model_reg.predict(val_x)
    xg_y = model_reg.predict(val_x)

    
    print("MSE:", metrics.mean_squared_error(val_y,xg_y))
    print("R2:", metrics.r2_score(val_y,xg_y))
    print("RMSE:", np.sqrt(metrics.mean_squared_error(val_y,xg_y)))
    print("MAPE:",mape(val_y,xg_y))
    print("MAE:",metrics.mean_absolute_error(val_y,xg_y))
xgb_reg()

MSE: 0.5289463407878161
R2: 0.7214432484304791
RMSE: 0.7272869727884695
MAPE: 7.958805319724198
MAE: 0.5196845147091814


# GA-XG预测

In [6]:
generations = 50   # 繁殖代数 400
pop_size = 50      # 种群数量  500
max_value = 10     # 基因中允许出现的最大值
chrom_length = 15  # 染色体长度  
pc = 0.6           # 交配概率  
pm = 0.01          # 变异概率  
results = []       # 存储每一代的最优解，N个三元组（auc最高值, n_estimators, max_depth）  
fit_value = []     # 个体适应度  
fit_mean = []      # 平均适应度 
random_seed = 20
cons_value = 0.19 / 31 # (0.20-0.01）/ (32 - 1)


dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x)


def xgboostModel(tree_num, eta, max_depth, min_child_weight,random_seed):
    
    params = {
#         'booster': 'gbtree',
#         'objective': "reg:squarederror",
#         'eval_metric': 'auc',
        'eta': eta,  # 0.02
        'max_depth': max_depth, # 8
        'min_child_weight': min_child_weight, # 3
#         'gamma': 0.1,
#         'subsample': 0.8,
#         'colsample_bytree': 0.8,
#         'lambda': 550,
#         'alpha': 19,
        'seed': randint(1,10000),
#         'nthread': 3
    }
    model = xgb.train(params, dtrain, num_boost_round=tree_num)
    predict_y = model.predict(dval, iteration_range = (0,0))
    mse = 1 / mean_squared_error(val_y, predict_y)
    return mse

# Step1:对参数进行编码
def geneEncoding(pop_size, chrom_length):  
    pop = [[]]
    for i in range(pop_size):
        temp = []
        for j in range(chrom_length):
            temp.append(random.randint(0, 1))
        pop.append(temp)
    return pop[1:]

# Step 2:计算个体的目标函数值
def cal_obj_value(pop):
    objvalue = []
    variable = decodechrom(pop)
    for i in range(len(variable)):
        tempVar = variable[i]
 
        tree_num_value = (tempVar[0] + 1)* 10
        eta_value = 0.01 + tempVar[1] * cons_value
        max_depth_value = 3 + tempVar[2]
        min_child_weight_value = 1 + tempVar[3]
 
        mseValue = xgboostModel(tree_num_value, eta_value, max_depth_value, min_child_weight_value, random_seed)
        objvalue.append(mseValue)
    return objvalue 
 
# 对每个个体进行解码
def decodechrom(pop):
    variable = []
    for i in range(len(pop)):
        res = []
        
        # 计算第一个变量值，即 0101->10(逆转)
        temp1 = pop[i][0:4]
        v1 = 0
        for i1 in range(4):
            v1 += temp1[i1] * (math.pow(2, i1))
        res.append(int(v1))
        
        # 计算第二个变量值
        temp2 = pop[i][4:9]
        v2 = 0
        for i2 in range(5):
            v2 += temp2[i2] * (math.pow(2, i2))
        res.append(int(v2))
 
        # 计算第三个变量值
        temp3 = pop[i][9:12]
        v3 = 0
        for i3 in range(3):
            v3 += temp3[i3] * (math.pow(2, i3))
        res.append(int(v3))
 
        # 计算第四个变量值
        temp4 = pop[i][12:15]
        v4 = 0
        for i4 in range(3):
            v4 += temp4[i4] * (math.pow(2, i4))
        res.append(int(v4))
 
        variable.append(res)
    return variable            # 500*4

# Step 3:计算个体的适应值，即最大值，淘汰负值
def calfitvalue(obj_value):
    fit_value = []
    temp = 0.0
    Cmin = 0
    for i in range(len(obj_value)):
        if(obj_value[i] + Cmin > 0):
            temp = Cmin + obj_value[i]
        else:
            temp = 0.0
        fit_value.append(temp)
    return fit_value
 
# Step 4:找出适应函数值中最大值,和对应的个体
def best(pop, fit_value):
    best_individual = pop[0]
    best_fit = fit_value[0]
    for i in range(1, len(pop)):
        if(fit_value[i] > best_fit):
            best_fit = fit_value[i]
            best_individual = pop[i]
    return [best_individual, best_fit]
 
# Step 5:每次繁殖，将最好的结果记录下来(将二进制转化为十进制)
def b2d(best_individual):
    # 计算第一个变量值
    temp1 = best_individual[0:4]
    v1 = 0
    for i1 in range(4):
        v1 += temp1[i1] * (math.pow(2, i1))
    v1 = (v1 + 1) * 10
    
    # 计算第二个变量值
    temp2 = best_individual[4:9]
    v2 = 0
    for i2 in range(5):
        v2 += temp2[i2] * (math.pow(2, i2))
    v2 = 0.01 + v2 * cons_value
 
    # 计算第三个变量值
    temp3 = best_individual[9:12]
    v3 = 0
    for i3 in range(3):
        v3 += temp3[i3] * (math.pow(2, i3))
    v3 = 3 + v3
 
    # 计算第四个变量值
    temp4 = best_individual[12:15]
    v4 = 0
    for i4 in range(3):
        v4 += temp4[i4] * (math.pow(2, i4))
    v4 = 1 + v4
 
    return int(v1), float(v2), int(v3), int(v4)

# Step 6:自然选择（轮盘赌算法）
def selection(pop, fit_value):
    # 计算每个适应值的概率
    new_fit_value = []
    total_fit = sum(fit_value)
    for i in range(len(fit_value)):
        new_fit_value.append(fit_value[i] / total_fit )
        
    # 计算每个适应值的累积概率
    cumsum(new_fit_value)
    
    # 生成随机浮点数序列
    ms = []
    pop_len = len(pop)
    for i in range(pop_len):
        ms.append(random.random())
    ms.sort()
    # 轮盘赌算法
    fitin = 0
    newin = 0
    newpop = pop
    while newin < pop_len:
        if(ms[newin] < new_fit_value[fitin]):
            newpop[newin] = pop[fitin]
            newin = newin + 1
        else:
            fitin = fitin + 1
    pop = newpop

    
# 求适应值的总和
def sum(fit_value):
    total = 0
    for i in range(len(fit_value)):
        total += fit_value[i]
    return total
 
# 计算累积概率
def cumsum(fit_value):
    temp=[]
    for i in range(len(fit_value)):
        t = 0
        j = 0
        while(j <= i):
            t += fit_value[j]
            j = j + 1
        temp.append(t)
    for i in range(len(fit_value)):
        fit_value[i]=temp[i]

        
# Step 7:交叉繁殖
def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2
 
 
# Step 8:基因突变
def mutation(pop, pm):
    px = len(pop)
    py = len(pop[0])
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0, py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1


def generAlgo(generations):
    pop = geneEncoding(pop_size, chrom_length)
    print(str(generations)+" start...")
    for i in range(generations):
        obj_value = cal_obj_value(pop) # 计算目标函数值
        fit_value = calfitvalue(obj_value) #计算个体的适应值
        [best_individual, best_fit] = best(pop, fit_value) #选出最好的个体和最好的函数值
        v1, v2, v3, v4 = b2d(best_individual)
        results.append([best_fit, v1, v2, v3, v4]) #每次繁殖，将最好的结果记录下来
        selection(pop, fit_value) #自然选择，淘汰掉一部分适应性低的个体
        crossover(pop, pc) #交叉繁殖
        mutation(pop, pc) #基因突变
    results.sort()
    print(results[-1])
    best_v1,best_v2,best_v3,best_v4 = results[-1][1],results[-1][2],results[-1][3],results[-1][4]
    print(best_v1,best_v2,best_v3,best_v4)
    return best_v1,best_v2,best_v3,best_v4

best_v1,best_v2,best_v3,best_v4,= generAlgo(50)

50 start...
[2.1406397568416513, 160, 0.06516129032258064, 7, 7]
160 0.06516129032258064 7 7


In [18]:
def gaxg_reg():
    model = xgb.XGBRegressor(n_estimators = best_v1, learning_rate=best_v2, max_depth=best_v3, 
                             min_child_weight=best_v4) # 遗传算法搜寻的最优参数
    model.fit(train_x, train_y)
    predict_y = model.predict(val_x)
    gaxg_y = model.predict(val_x)
    print("MSE:", metrics.mean_squared_error(val_y,gaxg_y))
    print("R2:", metrics.r2_score(val_y,gaxg_y))
    print("RMSE:", np.sqrt(metrics.mean_squared_error(val_y,gaxg_y)))
    print("MAPE:",mape(val_y,gaxg_y))
    print("MAE:",metrics.mean_absolute_error(val_y,gaxg_y))
    return gaxg_y
gaxg_y = gaxg_reg()

MSE: 0.4671500642758419
R2: 0.753986757510491
RMSE: 0.6834837703090263
MAPE: 7.572291374608847
MAE: 0.49400995055370517


# RF 预测

In [19]:
def rf_reg():
    rfr = RandomForestRegressor()
    rfr.fit(train_x,train_y)
    rf_y = rfr.predict(val_x)
    print("MSE:", metrics.mean_squared_error(val_y,rf_y))
    print("R2:", metrics.r2_score(val_y,rf_y))
    print("RMSE:", np.sqrt(metrics.mean_squared_error(val_y,rf_y)))
    print("MAPE:",mape(val_y,rf_y))
    print("MAE:",metrics.mean_absolute_error(val_y,rf_y))
rf_reg()

MSE: 0.48570253356769405
R2: 0.7442165498712161
RMSE: 0.696923621043005
MAPE: 7.836360635122299
MAE: 0.5104949580890049


# GA-RF预测

In [9]:
generations = 200   # 繁殖代数 200
pop_size = 200      # 种群数量  200
max_value = 10      # 基因中允许出现的最大值  
chrom_length = 16    # 染色体长度  
pc = 0.6            # 交配概率  
pm = 0.01           # 变异概率  
results = [[]]      # 存储每一代的最优解，N个三元组（auc最高值, n_estimators, max_depth）  
fit_value = []      # 个体适应度  
fit_mean = []       # 平均适应度 
pop = [[0, 1, 0, 1, 0, 1, 0, 1] for i in range(pop_size)] # 初始化种群中所有个体的基因初始序列
 
 

def randomForest(n_estimators_value, max_depth_value):
  
    rf = RandomForestRegressor(n_estimators=n_estimators_value,
                                max_depth=max_depth_value,
                                n_jobs=2)
    rf.fit(train_x, train_y)  # 训练分类器
    predict_test = rf.predict(val_x)
    mse = 1 / mean_squared_error(val_y, predict_test)
    # auc = metrics.auc(val_y,predict_test)
    return mse
 

# Step 1 : 对参数进行编码（用于初始化基因序列，可以选择初始化基因序列，本函数省略）
def geneEncoding(pop_size, chrom_length):  
    pop = [[]]
    for i in range(pop_size):
        temp = []
        for j in range(chrom_length):
            temp.append(random.randint(0, 1))
        pop.append(temp)
    return pop[1:]
 
# Step 2 : 计算个体的目标函数值
def cal_obj_value(pop):
    objvalue = []
    variable = decodechrom(pop)
    for i in range(len(variable)):
        tempVar = variable[i]
        n_estimators_value = (tempVar[0] + 1) * 10
        max_depth_value = tempVar[1] + 1
        mseValue = randomForest(n_estimators_value, max_depth_value)
        objvalue.append(mseValue)
    return objvalue #目标函数值objvalue[m] 与个体基因 pop[m] 对应 
 
 
 
# 对每个个体进行解码，并拆分成单个变量，返回 n_estimators 和 max_depth
def decodechrom(pop):
    variable = []
    n_estimators_value = []
    max_depth_value = []
    for i in range(len(pop)):
        res = []
        
        # 计算第一个变量值，即 0101->10(逆转)
        temp1 = pop[i][0:4]
        preValue = 0
        for pre in range(4):
            preValue += temp1[pre] * (math.pow(2, pre))
        res.append(int(preValue))
        
        # 计算第二个变量值
        temp2 = pop[i][4:8]
        aftValue = 0
        for aft in range(4):
            aftValue += temp2[aft] * (math.pow(2, aft))
        res.append(int(aftValue))
        variable.append(res)
    return variable
 
 
 
# Step 3: 计算个体的适应值（计算最大值，于是就淘汰负值就好了）
def calfitvalue(obj_value):
    fit_value = []
    temp = 0.0
    Cmin = 0
    for i in range(len(obj_value)):
        if(obj_value[i] + Cmin > 0):
            temp = Cmin + obj_value[i]
        else:
            temp = 0.0
        fit_value.append(temp)
    return fit_value
 
 
 
# Step 4: 找出适应函数值中最大值，和对应的个体
def best(pop, fit_value):
    best_individual = pop[0]
    best_fit = fit_value[0]
    for i in range(1, len(pop)):
        if(fit_value[i] > best_fit):
            best_fit = fit_value[i]
            best_individual = pop[i]
    return [best_individual, best_fit]
 
 
 
# Step 5: 每次繁殖，将最好的结果记录下来(将二进制转化为十进制)
def b2d(best_individual):
    temp1 = best_individual[0:4]
    preValue = 0
    for pre in range(4):
        preValue += temp1[pre] * (math.pow(2, pre))
    preValue = preValue + 1
    preValue = preValue * 10
    
    # 计算第二个变量值
    temp2 = best_individual[4:8]
    aftValue = 0
    for aft in range(4):
        aftValue += temp2[aft] * (math.pow(2, aft))
    aftValue = aftValue + 1
    return int(preValue), int(aftValue)
 
 
 
# Step 6: 自然选择（轮盘赌算法）
def selection(pop, fit_value):
    # 计算每个适应值的概率
    new_fit_value = []
    total_fit = sum(fit_value)
    for i in range(len(fit_value)):
        new_fit_value.append(fit_value[i] / total_fit)
    # 计算每个适应值的累积概率
    cumsum(new_fit_value)
    # 生成随机浮点数序列
    ms = []
    pop_len = len(pop)
    for i in range(pop_len):
        ms.append(random.random())
    # 对生成的随机浮点数序列进行排序
    ms.sort()
    # 轮盘赌算法（选中的个体成为下一轮，没有被选中的直接淘汰，被选中的个体代替）
    fitin = 0
    newin = 0
    newpop = pop
    while newin < pop_len:
        if(ms[newin] < new_fit_value[fitin]):
            newpop[newin] = pop[fitin]
            newin = newin + 1
        else:
            fitin = fitin + 1
    pop = newpop
 
 
# 求适应值的总和
def sum(fit_value):
    total = 0
    for i in range(len(fit_value)):
        total += fit_value[i]
    return total
 
# 计算累积概率
def cumsum(fit_value):
    temp=[]
    for i in range(len(fit_value)):
        t = 0
        j = 0
        while(j <= i):
            t += fit_value[j]
            j = j + 1
        temp.append(t)
    for i in range(len(fit_value)):
        fit_value[i]=temp[i]
 
 
# Step 7: 交叉繁殖
def crossover(pop, pc): #个体间交叉，实现基因交换
    poplen = len(pop)
    for i in range(poplen - 1):
        if(random.random() < pc):
            cpoint = random.randint(0,len(pop[0]))
            temp1 = []
            temp2 = []
            temp1.extend(pop[i][0 : cpoint])
            temp1.extend(pop[i+1][cpoint : len(pop[i])])
            temp2.extend(pop[i+1][0 : cpoint])
            temp2.extend(pop[i][cpoint : len(pop[i])])
            pop[i] = temp1
            pop[i+1] = temp2
 
 
 
 
# Step 8: 基因突变
def mutation(pop, pm):
    px = len(pop)
    py = len(pop[0])
    for i in range(px):
        if(random.random() < pm):
            mpoint = random.randint(0,py-1)
            if(pop[i][mpoint] == 1):
                pop[i][mpoint] = 0
            else:
                pop[i][mpoint] = 1
 
 
for i in range(generations):
    print("第 " + str(i) + " 代开始繁殖......")
    obj_value = cal_obj_value(pop) # 计算目标函数值
    # print(obj_value)
    fit_value = calfitvalue(obj_value) #计算个体的适应值
    # print(fit_value)
    [best_individual, best_fit] = best(pop, fit_value) #选出最好的个体和最好的函数值
    # print("best_individual: "+ str(best_individual))
    temp_n_estimator, temp_max_depth = b2d(best_individual)
    results.append([best_fit, temp_n_estimator, temp_max_depth]) #每次繁殖，将最好的结果记录下来
    print(str(best_individual) + " " + str(best_fit))
    selection(pop, fit_value) #自然选择，淘汰掉一部分适应性低的个体
    crossover(pop, pc) #交叉繁殖
    mutation(pop, pc) #基因突变
# print(results)
results.sort()
print(results[-1])
n_estimators_value,max_depth_value = results[-1][1],results[-1][2]
results

第 0 代开始繁殖......
[0, 1, 0, 1, 0, 1, 0, 1] 2.078966579513015
第 1 代开始繁殖......
[0, 1, 0, 1, 1, 1, 0, 1] 2.110100641674262
第 2 代开始繁殖......
[0, 1, 0, 1, 0, 1, 1, 1] 2.112555911000954
第 3 代开始繁殖......
[0, 1, 0, 0, 0, 0, 1, 1] 2.1586349587640212
第 4 代开始繁殖......
[0, 1, 0, 1, 0, 0, 1, 1] 2.1128210144461965
第 5 代开始繁殖......
[0, 1, 1, 1, 1, 1, 0, 1] 2.102810894452407
第 6 代开始繁殖......
[0, 1, 0, 0, 1, 0, 1, 1] 2.123803965424284
第 7 代开始繁殖......
[1, 1, 0, 1, 0, 0, 1, 1] 2.1134765113341363
第 8 代开始繁殖......
[0, 1, 1, 1, 0, 0, 1, 1] 2.1097918664593402
第 9 代开始繁殖......
[0, 0, 0, 1, 0, 1, 1, 1] 2.116516501976821
第 10 代开始繁殖......
[1, 0, 0, 1, 1, 0, 1, 1] 2.137001157981791
第 11 代开始繁殖......
[1, 0, 0, 1, 1, 0, 1, 1] 2.1256054100357087
第 12 代开始繁殖......
[0, 0, 0, 1, 0, 0, 1, 1] 2.124385963361627
第 13 代开始繁殖......
[0, 1, 0, 1, 1, 0, 1, 1] 2.1178801294340768
第 14 代开始繁殖......
[0, 1, 0, 0, 1, 0, 1, 1] 2.134514827077259
第 15 代开始繁殖......
[1, 1, 0, 0, 1, 1, 1, 1] 2.159736555576737
第 16 代开始繁殖......
[0, 1, 1, 0, 1, 0, 1, 1] 2.

[[],
 [2.078966579513015, 110, 11],
 [2.0964219914016673, 40, 13],
 [2.0971745465516682, 110, 15],
 [2.10169995679126, 160, 13],
 [2.102810894452407, 150, 12],
 [2.1053865687947075, 120, 16],
 [2.1060483257282647, 160, 14],
 [2.1079525623233697, 160, 13],
 [2.1097918664593402, 150, 13],
 [2.110100641674262, 110, 12],
 [2.110732571154037, 40, 16],
 [2.110959540877626, 140, 15],
 [2.111072648610987, 160, 14],
 [2.112555911000954, 110, 15],
 [2.1128210144461965, 110, 13],
 [2.1134765113341363, 120, 13],
 [2.1154389468825925, 160, 14],
 [2.116381171664889, 160, 14],
 [2.116516501976821, 90, 15],
 [2.1178801294340768, 110, 14],
 [2.1183402697860716, 70, 14],
 [2.1230670340193263, 140, 15],
 [2.1236440769697813, 140, 14],
 [2.123803965424284, 30, 14],
 [2.1239857115367786, 120, 15],
 [2.124241890933406, 160, 13],
 [2.124385963361627, 90, 13],
 [2.1256054100357087, 100, 14],
 [2.125956444732616, 80, 16],
 [2.128139191257898, 40, 16],
 [2.128157952292768, 80, 16],
 [2.1295959083795952, 110, 16

In [14]:
def garf_reg():
    rf = RandomForestRegressor(n_estimators = 100, max_depth = 15)
    rf.fit(train_x,train_y)
    garf_y = rf.predict(val_x)
    print("MSE:", metrics.mean_squared_error(val_y,garf_y))
    print("R2:", metrics.r2_score(val_y,garf_y))
    print("RMSE:", np.sqrt(metrics.mean_squared_error(val_y,garf_y)))
    print("MAPE:",mape(val_y,garf_y))
    print("MAE:",metrics.mean_absolute_error(val_y,garf_y))
    return garf_y
garf_y = garf_reg()

MSE: 0.4775765776194676
R2: 0.7484958873347549
RMSE: 0.6910691554536836
MAPE: 7.783765375326247
MAE: 0.5083751909159986


# GA-XG-RF

In [15]:
DNA_SIZE = 24
POP_SIZE = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.005
N_GENERATIONS = 50
W_XG_BOUND = [0, 1]
W_RF_BOUND = [0, 1]
test_y = np.array(val_y)


def F(W_XG):
    
    y_t = test_y
    xg = gaxg_y
    rf = garf_y
    W_XG = W_XG.reshape(100,1)
    W_RF = (1-W_XG).reshape(100,1)
    return ((W_XG * xg + W_RF * rf -y_t) ** 2).sum(axis =1).reshape(100,1)

def get_fitness(pop): 
    W_XG,W_RF = translateDNA(pop)
    pred = F(W_XG)
    fitness = -(pred - np.max(pred)) + 1e-3
    return  fitness.reshape(100,)#减去最小的适应度是为了防止适应度出现负数，通过这一步fitness的范围为[0, np.max(pred)-np.min(pred)],最后在加上一个很小的数防止出现为0的适应度


def translateDNA(pop): #pop表示种群矩阵，一行表示一个二进制编码表示的DNA，矩阵的行数为种群数目
    W_XG_pop = pop[:,1::2]#奇数列表示X
    W_RF_pop = pop[:,::2] #偶数列表示y

    #pop:(POP_SIZE,DNA_SIZE)*(DNA_SIZE,1) --> (POP_SIZE,1)
    W_XG = W_XG_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(W_XG_BOUND[1]-W_XG_BOUND[0])+W_XG_BOUND[0]
    W_RF = W_RF_pop.dot(2**np.arange(DNA_SIZE)[::-1])/float(2**DNA_SIZE-1)*(W_RF_BOUND[1]-W_RF_BOUND[0])+W_RF_BOUND[0]
    return W_XG,W_RF

def crossover_and_mutation(pop, CROSSOVER_RATE = 0.8):
    new_pop = []
    for father in pop:		#遍历种群中的每一个个体，将该个体作为父亲
        child = father		#孩子先得到父亲的全部基因（这里我把一串二进制串的那些0，1称为基因）
        if np.random.rand() < CROSSOVER_RATE:            #产生子代时不是必然发生交叉，而是以一定的概率发生交叉
            mother = pop[np.random.randint(POP_SIZE)]    #再种群中选择另一个个体，并将该个体作为母亲
            cross_points = np.random.randint(low=0, high=DNA_SIZE*2)  #随机产生交叉的点
            child[cross_points:] = mother[cross_points:] #孩子得到位于交叉点后的母亲的基因
        mutation(child)	#每个后代有一定的机率发生变异
        new_pop.append(child)

    return new_pop

def mutation(child, MUTATION_RATE=0.03):
    if np.random.rand() < MUTATION_RATE:                 #以MUTATION_RATE的概率进行变异
        mutate_point = np.random.randint(0, DNA_SIZE*2)  #随机产生一个实数，代表要变异基因的位置
        child[mutate_point] = child[mutate_point]^1 #    将变异点的二进制为反转

def select(pop, fitness):    # nature selection wrt pop's fitness
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,
                           p=(fitness)/(fitness.sum()) )
    return pop[idx]

def print_info(pop):
    fitness = get_fitness(pop)
    max_fitness_index = np.argmax(fitness)

    print("max_fitness:", fitness[max_fitness_index])
    W_XG,W_RF = translateDNA(pop)
    print("最优的基因型：", pop[max_fitness_index])
    print("(W_XG),(W_RF):", (W_XG[max_fitness_index]),(1-W_XG[max_fitness_index]))
    return ( W_XG[max_fitness_index],1-W_XG[max_fitness_index])


pop = np.random.randint(2, size=(POP_SIZE, DNA_SIZE*2)) #matrix (POP_SIZE, DNA_SIZE)
for _ in range(N_GENERATIONS):#迭代N代
    W_XG,W_RF = translateDNA(pop)
    pop = np.array(crossover_and_mutation(pop, CROSSOVER_RATE))
    fitness = get_fitness(pop)
    pop = select(pop, fitness) #选择生成新的种群
print_info(pop)
max_fitness_index = np.argmax(fitness)
W_XG,W_RF = translateDNA(pop)
xg_weight,rf_weight = W_XG[max_fitness_index],1-W_XG[max_fitness_index]


max_fitness: 0.01682650847362288
最优的基因型： [0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 1 1 1
 0 1 1 1 0 1 1 1 1 1 0]
(W_XG),(W_RF): 0.6072820787001895 0.39271792129981054


# 组合结果

In [20]:
gaxgrf_y = xg_weight * gaxg_y + rf_weight *  garf_y
gaxgrf_y
print("MSE:", metrics.mean_squared_error(val_y,gaxgrf_y))
print("R2:", metrics.r2_score(val_y,gaxgrf_y))
print("RMSE:", np.sqrt(metrics.mean_squared_error(val_y,gaxgrf_y)))
print("MAPE:",mape(val_y,gaxgrf_y))
print("MAE:",metrics.mean_absolute_error(val_y,gaxgrf_y))

MSE: 0.4598608853403454
R2: 0.7578254266708765
RMSE: 0.6781304338697279
MAPE: 7.566319422230923
MAE: 0.49354748945336047
